In [1]:
import pandas as pd

from helper import *
from snp_func import *

# Do assignments
a = assign_var('snp')
for v in a:
    exec(v)

from ib_insync import *
util.startLoop()
ib = get_connected('snp', 'live')

# Code to set up harvest quotes
# ______________________________
p = util.df(ib.portfolio()) # portfolio table

# extract option contract info from portfolio table
dfp = pd.concat([p, util.df([c for c in p.contract])[util.df([c for c in p.contract]).columns[:7]]], axis=1).iloc[:, 1:]
dfp = dfp.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})

dfp = dfp.assign(dte=[get_dte(d) for d in dfp.expiration])

# get the latest prices
contracts = ib.qualifyContracts(*[Contract(conId=c) for c in dfp.conId])

tickers = ib.reqTickers(*contracts)
optPrices = {t.contract.conId: t.marketPrice() for t in tickers} # {conId: optPrice}

# make the expPrice to be the price minimum of df.marketPrice*0.9 and hvstPrice()
dfp1 = dfp.assign(hvstPrice=[get_prec(min(hvstPricePct(d),mp*0.9),0.01) for d, mp in zip(dfp.dte, dfp.marketPrice)])
dfp1 = dfp1.loc[:, ['position', 'marketPrice', 'averageCost', 'secType', 'conId', 'symbol', 'expiration', 'strike', 'right', 'dte', 'hvstPrice']]

dfp1 = dfp1.assign(order=[LimitOrder(action='BUY', 
                                     totalQuantity=-position, lmtPrice=hvstPrice) 
                          for position, hvstPrice in zip(dfp1.position, dfp1.hvstPrice)])

In [2]:
dfp1

,position,marketPrice,averageCost,secType,conId,symbol,expiration,strike,right,dte,hvstPrice,order
0,-2.0,0.003367,6.38405,OPT,334998765,AIG,20190517,35.0,P,10,0.00,"LimitOrder(action='BUY', totalQuantity=2.0, lm..."
1,-1.0,0.082172,7.20400,OPT,363488157,CVX,20190607,100.0,P,31,0.07,"LimitOrder(action='BUY', totalQuantity=1.0, lm..."
2,-2.0,0.500524,40.40335,OPT,359703281,DOW,20190621,47.5,P,45,0.26,"LimitOrder(action='BUY', totalQuantity=2.0, lm..."
3,-2.0,0.315390,6.90405,OPT,361625507,DWDP,20190517,32.0,P,10,0.28,"LimitOrder(action='BUY', totalQuantity=2.0, lm..."
4,-1.0,0.471037,16.90380,OPT,315245939,MMM,20190621,155.0,P,45,0.26,"LimitOrder(action='BUY', totalQuantity=1.0, lm..."
5,-1.0,0.108507,10.20400,OPT,316304932,MO,20190621,40.0,P,45,0.10,"LimitOrder(action='BUY', totalQuantity=1.0, lm..."
6,-1.0,0.082561,6.20410,OPT,315952800,PM,20190621,60.0,P,45,0.07,"LimitOrder(action='BUY', totalQuantity=1.0, lm..."
7,-1.0,0.132827,10.20400,OPT,362480853,WFC,20190607,43.0,P,31,0.12,"LimitOrder(action='BUY', totalQuantity=1.0, lm..."


In [ ]:
df_hvstTrades = pd.concat([pd.DataFrame(harvestTrades), util.df(harvestTrades)], axis=1)
df_hvstTrades.columns=[['trade']+list(df_hvstTrades)[1:]]
df_hvstTrades.to_pickle(fspath+'_hvstTradesDone.pickle')

In [ ]:
df_hvstTrades

In [4]:
# Code to set up sowing orders
# ____________________________
dft = pd.read_pickle(fspath+'_targetopts.pickle')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
grp_opts(dft)

,optId,symbol,right,expiration,dte,strike,undPrice,lo52,hi52,Fall,Rise,loFall,hiRise,Std,loStd,hiStd,lotsize,optPrice,optMargin,rom,remqty,expPrice,expRom
0,361264805,ABBV,P,20190524,17,74.0,78.71,75.77,125.86,33.230,27.82,45.480,106.53,1.980324,74.749352,84.650972,100,0.29,829,0.751082,6,0.319,0.826190
1,361264760,ABBV,P,20190524,17,73.5,78.71,75.77,125.86,33.230,27.82,45.480,106.53,1.980324,74.749352,84.650972,100,0.25,793,0.676879,6,0.280,0.758104
2,361509387,ABBV,P,20190517,10,74.5,78.71,75.77,125.86,29.690,27.20,49.020,105.91,1.980324,74.749352,84.650972,100,0.18,854,0.769321,6,0.198,0.846253
3,316297786,ABT,P,20190621,45,52.5,78.69,54.32,80.74,9.480,13.47,69.210,92.16,4.123154,70.443691,91.059463,100,0.01,305,0.026594,6,0.280,0.744627
4,315853195,ACN,P,20190621,45,130.0,176.98,132.63,183.35,35.740,32.27,141.240,209.25,13.381103,150.217794,217.123309,100,0.05,956,0.042422,2,0.880,0.746629
5,315853188,ACN,P,20190621,45,125.0,176.98,132.63,183.35,35.740,32.27,141.240,209.25,13.381103,150.217794,217.123309,100,0.05,841,0.048223,2,0.780,0.752279
6,337639876,AIG,P,20190517,10,36.0,47.11,36.16,65.05,10.950,5.71,36.160,52.82,1.427637,44.254727,51.392910,100,0.07,287,0.890244,8,0.077,0.979268
7,334998765,AIG,P,20190517,10,35.0,47.11,36.16,65.05,10.950,5.71,36.160,52.82,1.427637,44.254727,51.392910,100,0.07,264,0.967803,8,0.077,1.064583
8,337639862,AIG,P,20190517,10,34.0,47.11,36.16,65.05,10.950,5.71,36.160,52.82,1.427637,44.254727,51.392910,100,0.04,241,0.605809,8,0.070,1.060166
9,364081993,AMGN,P,20190614,38,162.5,177.31,163.31,210.19,32.980,32.98,144.330,210.29,7.137436,163.035128,198.722309,100,1.63,1731,0.904482,2,1.793,0.994930


In [5]:
# qualify contracts and prepare to order
contracts = [Contract(conId=c) for c in dft.optId]
cblks = [contracts[i: i+blk] for i in range(0, len(contracts), blk)]
qc = [c for q in [ib.qualifyContracts(*c) for c in cblks] for c in q]

sell_order = [LimitOrder(action='SELL', totalQuantity=1, lmtPrice=expPrice) for q, expPrice in zip(dft.lotsize, dft.expPrice)]

co = list(zip(qc, sell_order))
coblks = [co[i: i+blk] for i in range(0, len(co), blk)]

util.logToConsole()

Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests


In [ ]:
[(co[0], co[1]) for coblk in coblks for co in coblk]

In [ ]:
df_trades = pd.concat([pd.DataFrame(trades), util.df(trades)], axis=1)
df_trades.columns=[['trade']+list(df_trades)[1:]]
df_trades.to_pickle(fspath+'_tradesDone.pickle')

df_trades.head()

In [ ]:
# Execute the trades
def xcute(ib, fspath):
    '''Executes trades
    Args:
       (ib) as connection object
       (fspath) as string with path to pickles
    Returns:
       None. But pickles _tradesdone.pickle'''
    
    # read from the targets
    dft = pd.read_pickle(fspath+'_targetopts.pickle')